In [2]:
import pyspark
import os.path

In [3]:
sc = pyspark.SparkContext(master="local[1]",appName='metyis_challenge')

In [4]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [5]:
# Vamos a definir un schema personalizado para poder tratar las fechas convenientemente
from pyspark.sql.types import *

bixiSchema = StructType([ \
    StructField("_c0", DoubleType(), True), \
    StructField("start_date", DateType(), True), \
    StructField("start_station_code", StringType(), True), \
    StructField("end_date", DateType(), True), \
    StructField("end_station_code", StringType(), True), \
    StructField("duration_sec", DoubleType(), True), \
    StructField("is_member", BooleanType(), True)])

In [6]:
bixiDF = sqlContext.read.format('csv').options(header='true', delimiter = ',').load('./data/OD_2014.csv', schema = bixiSchema)
print(bixiDF.count())
bixiDF.take(3)

3136276


[Row(_c0=0.0, start_date=datetime.date(2014, 4, 15), start_station_code='6209', end_date=datetime.date(2014, 4, 15), end_station_code='6436', duration_sec=1061.0, is_member=None),
 Row(_c0=1.0, start_date=datetime.date(2014, 4, 15), start_station_code='6214', end_date=datetime.date(2014, 4, 15), end_station_code='6248', duration_sec=615.0, is_member=None),
 Row(_c0=2.0, start_date=datetime.date(2014, 4, 15), start_station_code='6164', end_date=datetime.date(2014, 4, 15), end_station_code='6216', duration_sec=1031.0, is_member=None)]

In [7]:
bixiDF.describe().show()

+-------+-----------------+------------------+------------------+-----------------+
|summary|              _c0|start_station_code|  end_station_code|     duration_sec|
+-------+-----------------+------------------+------------------+-----------------+
|  count|          3136276|           3136276|           3136276|          3136276|
|   mean|        1568137.5|  6237.94442134557| 6231.226863643378|789.7391029998636|
| stddev|905365.0407640373|273.04065261969197|281.75890464523695|627.6714220519676|
|    min|              0.0|             10002|             10002|             61.0|
|    max|        3136275.0|              6929|              6929|           7195.0|
+-------+-----------------+------------------+------------------+-----------------+



In [22]:
display(bixiDF)

DataFrame[_c0: string, start_date: string, start_station_code: string, end_date: string, end_station_code: string, duration_sec: string, is_member: string]

In [23]:
year = '2014'

In [20]:
sqlContext.sql('DROP TABLE IF EXISTS metyis_bixi_travels')
sqlContext.registerDataFrameAsTable(bixiDF, 'metyis_bixi_travels')